In [ ]:
import numpy as np
import pandas as pd

def model_shock(
    df: pd.DataFrame,
    model,                              # fitted statsmodels GLM
    shocks: list,                       # e.g. [-0.2, -0.1, 0, 0.1, 0.2]
    vars_bounds: dict,                  # e.g. {'var1':[0,1], 'var2':[None, 5]}
    agg_func=np.mean                    # how to summarise predictions
) -> pd.DataFrame:
    """
    One-at-a-time percentage shocks on selected variables.

    Parameters
    ----------
    df : DataFrame            – data used for prediction
    model : statsmodels GLM    – fitted model with .predict()
    shocks : list[float]       – decimal shocks (±0.1 ⇒ ±10 %)
    vars_bounds : dict         – {var: [lower, upper]} (None means no bound)
    agg_func : callable        – statistic to report (default = mean)

    Returns
    -------
    DataFrame with columns:
        variable   – shocked variable name
        shock      – applied % change
        stat       – aggregate of predictions under shock
        delta      – absolute change vs baseline
        pct_change – % change vs baseline
    """
    # baseline
    baseline_pred   = model.predict(df)
    baseline_stat   = agg_func(baseline_pred)

    rows = []
    for var, bounds in vars_bounds.items():
        lower, upper = (bounds + [None, None])[:2] if isinstance(bounds, list) else (None, None)

        for s in shocks:
            # skip re-doing the baseline row if s == 0
            if s == 0 and rows:  
                continue

            dfi           = df.copy()
            dfi[var]      = dfi[var] * (1 + s)

            if lower is not None:
                dfi[var] = dfi[var].clip(lower=lower)
            if upper is not None:
                dfi[var] = dfi[var].clip(upper=upper)

            pred          = model.predict(dfi)
            stat          = agg_func(pred)
            delta         = stat - baseline_stat
            pct_change    = delta / baseline_stat if baseline_stat != 0 else np.nan

            rows.append({
                'variable':   var,
                'shock':      s,
                'stat':       stat,
                'delta':      delta,
                'pct_change': pct_change
            })

    return pd.DataFrame(rows)